In [38]:
import pandas as pd
import numpy as np
import statsmodels.stats.multicomp as multi 
import statsmodels.formula.api as smf


In [19]:
add_health = pd.read_csv("add_health.csv", low_memory=False)

In [43]:
# Data Management
# Sacred texts agreement question: 
#replace answers in sacred texts to nan
add_health['H1RE2'] = add_health['H1RE2'].replace(6, np.NaN)
add_health['H1RE2'] = add_health['H1RE2'].replace(8, np.NaN)
#if has no religion replace with the value disagree
add_health['H1RE2'] = add_health['H1RE2'].replace(7, 2)

#howm important is religion for you
add_health['H1RE4'] = add_health['H1RE4'].replace(6, np.NaN)
add_health['H1RE4'] = add_health['H1RE4'].replace(8, np.NaN)
#if has no religion replace with the value not important
add_health['H1RE4'] = add_health['H1RE4'].replace(7, 4)
add_health['H1RE4'] = add_health['H1RE4'].astype('category')

#how often do you pray
add_health['H1RE6'] = add_health['H1RE6'].replace(6, np.NaN)
add_health['H1RE6'] = add_health['H1RE6'].replace(8, np.NaN)
#if has no religion replace with the value never
add_health['H1RE6'] = add_health['H1RE6'].replace(7, 5)


# Handling response variable: how often did you skip full day in school without an excuse?
Bare in mind, that originally it's a string variable, because of "Skip" answer, which is marked as " "

In [44]:
add_health['SCHSKIP'] = pd.to_numeric(add_health['H1ED2'].replace(["996","997", "998", " "], np.NaN))

In [45]:
#choose only columns, that makes interest
selected_part = add_health[['H1RE2', 'H1RE4', 'H1RE6', 'H1PR5', 'H1PF2','H1ED2']]

# Statistical check
### Null-hypothesis is: religious adolescents skip the same amount of days in school as non-religious adolescents
Religion amongst adolescents is a categorical variable with 4 levels
Amount of skipped days is quantative variable

### Mean and Standard deviations for amount of skipped days grouped by religiosity 

In [52]:
print ('Mean value for amount of skipped days by religiosity level')

m1= sub.groupby('H1RE4').mean()
print (m1)

print ('standard deviations for amount of skipped days by religiosity level')
sd1 = sub.groupby('H1RE4').std()
print (sd1)


Mean value for amount of skipped days by religiosity level
        SCHSKIP
H1RE4          
1.0    1.171201
2.0    2.182361
3.0    3.140992
4.0    3.434951
standard deviations for amount of skipped days by religiosity level
         SCHSKIP
H1RE4           
1.0     5.672074
2.0     7.696166
3.0     9.995861
4.0    10.046981


### Ordinary least squared mathod for categorical variable with more than 2 levels

In [49]:
sub = add_health[['H1RE4', 'SCHSKIP']].dropna()
sub['H1RE4'] = sub['H1RE4'].astype('category')
model1 = smf.ols(formula='SCHSKIP ~ C(H1RE4)', data=sub)
results1 = model1.fit()
print (results1.summary())


                            OLS Regression Results                            
Dep. Variable:                SCHSKIP   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     26.96
Date:                Sat, 28 Mar 2020   Prob (F-statistic):           2.54e-17
Time:                        19:36:57   Log-Likelihood:                -21817.
No. Observations:                6347   AIC:                         4.364e+04
Df Residuals:                    6343   BIC:                         4.367e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           1.1712      0.143     

Based on p = 2.54e-17 Null-hypothesis can be rejected, which means, that adolescents, for whom religion is more important tend to skip less days in school without explanations. 

### Multiple comparisons

In [51]:
mc1 = multi.MultiComparison(sub['SCHSKIP'], sub['H1RE4'])
res1 = mc1.tukeyhsd()
print(res1.summary())

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
   1.0    2.0   1.0112  0.001  0.4564 1.5659   True
   1.0    3.0   1.9698  0.001  0.9147 3.0249   True
   1.0    4.0   2.2638  0.001  1.5572 2.9703   True
   2.0    3.0   0.9586 0.0986 -0.1135 2.0307  False
   2.0    4.0   1.2526  0.001  0.5209 1.9843   True
   3.0    4.0    0.294    0.9  -0.864 1.4519  False
---------------------------------------------------


Multiple comparisons shows that there's significant difference between amount of skipped days for those adolescents, who consider religion to be a very important important part of their lifes from all the other groups. And also there's a difference between those, for whom religion is fairly important from those, for whom it is not important at all. 